In [2]:

import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import cell2location
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42

/work/wjx/miniconda3/envs/cell2loc_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

results_folder = '/work/wjx/scRNA-seq/AIP/Spatial/result/'
ref_run_name = f'{results_folder}/reference_signatures'
run_name = f'{results_folder}/cell2location_map'


In [ ]:

adata_vis = sc.read_visium(
    '/work/wjx/scRNA-seq/AIP/Spatial/matrix/AIP20_35834/', count_file='filtered_feature_bc_matrix.h5',
    genome=None, library_id=None, load_images=True,
)  
adata_vis
adata_vis.obs['sample'] = list(adata_vis.uns['spatial'].keys())[0]
adata_vis.var
adata_vis.var['SYMBOL'] = adata_vis.var_names

In [ ]:


adata_vis.var['MT_gene'] = [gene.startswith('MT-') for gene in adata_vis.var['SYMBOL']]


adata_vis.obsm['MT'] = adata_vis[:, adata_vis.var['MT_gene'].values].X.toarray()
adata_vis = adata_vis[:, ~adata_vis.var['MT_gene'].values]
adata_vis

In [ ]:

adata_file = f"/work/wjx/scRNA-seq/AIP/Spatial/scref_downsample20240717.h5ad"
adata_ref = sc.read_h5ad(adata_file)
adata_ref

adata_ref.var

adata_ref.var['SYMBOL'] = adata_ref.var.index
from cell2location.utils.filtering import filter_genes
selected = filter_genes(adata_ref, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)
adata_ref = adata_ref[:, selected].copy()

In [ ]:
cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                        batch_key='orig.ident',
                        labels_key='celltype1',
                        
                       )

from cell2location.models import RegressionModel
mod = RegressionModel(adata_ref)

In [ ]:
# view anndata_setup as a sanity check
mod.view_anndata_setup()
mod.train(max_epochs=250)
mod.plot_history(20)
# In this section, we export the estimated cell abundance (summary of the posterior distribution).
adata_ref = mod.export_posterior(
    adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500}
)

# Save model
mod.save(f"{ref_run_name}", overwrite=True)

# Save anndata object with results
adata_file = f"{ref_run_name}/sc.h5ad"
adata_ref.write(adata_file)
adata_file

mod.plot_QC()


In [ ]:

adata_file = f"{ref_run_name}/sc.h5ad"
adata_ref = sc.read_h5ad(adata_file)
mod = cell2location.models.RegressionModel.load(f"{ref_run_name}", adata_ref)

# export estimated expression in each cluster
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver

In [ ]:
# find shared genes and subset both anndata and reference signatures
intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
inf_aver = inf_aver.loc[intersect, :].copy()
adata_vis.var_names_make_unique()
adata_vis = adata_vis[:, intersect].copy()
adata_vis

In [ ]:
# prepare anndata for cell2location model
cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key="sample")
# create and train the model
mod = cell2location.models.Cell2location(
    adata_vis, cell_state_df=inf_aver,
    # the expected average cell abundance: tissue-dependent
    # hyper-prior which can be estimated from paired histology:
    N_cells_per_location=20,
    # hyperparameter controlling normalisation of
    # within-experiment variation in RNA detection:
    detection_alpha=20
)
mod.view_anndata_setup()


In [ ]:
#use_gpu=True,
mod.train(max_epochs=30000,
          # train using full data (batch_size=None)
          batch_size=None,
          # use all data points in training because
          # we need to estimate cell abundance at all locations
          train_size=1,
         )
# plot ELBO loss history during training, removing first 100 epochs from the plot
mod.plot_history(100)
plt.legend(labels=['full data training'])
plt.savefig('/work/wjx/scRNA-seq/AIP/Spatial/full_data_training.pdf')
mod.plot_history(100)

In [ ]:
# In this section, we export the estimated cell abundance (summary of the posterior distribution).#, 'use_gpu': True
adata_vis = mod.export_posterior(
    adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs}
)

# Save model
mod.save(f"{run_name}", overwrite=True)

# mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)



In [ ]:
# Save anndata object with results
adata_file = f"{run_name}/sp.h5ad"
adata_vis.write(adata_file)
adata_file


In [5]:

adata_file = f"{run_name}/sp.h5ad"
adata_vis = sc.read_h5ad(adata_file)
#mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)
#mod.plot_QC()
adata_vis.obsm

AxisArrays with keys: MT, means_cell_abundance_w_sf, q05_cell_abundance_w_sf, q95_cell_abundance_w_sf, spatial, stds_cell_abundance_w_sf

In [7]:

#pd.DataFrame(adata_vis.obsm['q05_cell_abundance_w_sf']).to_csv("/work/wjx/spatial/IBD_spatial/figure/figure3/result/st_cell2location_res.csv")
# add 5% quantile, representing confident cell abundance, 'at least this amount is present',
# to adata.obs with nice names for plotting
adata_vis.obs[adata_vis.uns['mod']['factor_names']] = adata_vis.obsm['q05_cell_abundance_w_sf']
adata_vis.obs['array_row']=adata_vis.obs['array_row'].apply(int)
adata_vis.obs['array_col']=adata_vis.obs['array_col'].apply(int)
adata_vis.obsm['spatial']=adata_vis.obsm['spatial'].astype(int)
adata_vis.obs['in_tissue']=adata_vis.obs['in_tissue'].apply(int)
# select one slide
#from cell2location.utils import select_slide
#slide = select_slide(adata_vis, 'CD179_INF5')

In [ ]:
# plot in spatial coordinates
with mpl.rc_context({'axes.facecolor':  'black',
                     'figure.figsize': [4.5, 5]}):
    sc.pl.spatial(adata_vis, cmap='magma',
                  # show first 8 cell types
                  color=[
                           "CXCL9+ Macrophage"          
                       ],
                  ncols=4, size=1.3,
                  img_key='hires',
                  # limit color scale at 99.2% quantile of cell abundance
                  vmin=0, vmax='p99.2'
                 )
#plt.savefig('POSTN+Fibroblast.pdf')                 

In [ ]:
# Now we use cell2location plotter that allows showing multiple cell types in one panel
from cell2location.plt import plot_spatial


# select up to 6 clusters
clust_labels = ["ABC(IgD-)","Tfh",]
clust_col = ['' + str(i) for i in clust_labels] # in case column names differ from labels

#slide = select_slide(adata_vis, 'V1_Human_Lymph_Node')

with mpl.rc_context({'figure.figsize': (8, 8)}):
        plot_spatial(
        adata=adata_vis,
        # labels to show on a plot
        color=clust_col, labels=clust_labels,
        show_img=True,
        reorder_cmap=(1,2),
        crop_x=(130,1250),
        crop_y=(420,1550),
        # 'fast' (white background) or 'dark_background'
        style='fast',
        # limit color scale at 99.2% quantile of cell abundance
        max_color_quantile=0.992,
        # size of locations (adjust depending on figure size)
        circle_diameter=6,
        colorbar_position='right'
    )    
plt.savefig('Figure5b.pdf')  

In [12]:
# Now we use cell2location plotter that allows showing multiple cell types in one panel
from cell2location.plt import plot_spatial


# select up to 6 clusters
clust_labels = ["ABC(IgD-)","CXCL9+ Macrophage",]
clust_col = ['' + str(i) for i in clust_labels] # in case column names differ from labels

#slide = select_slide(adata_vis, 'V1_Human_Lymph_Node')

with mpl.rc_context({'figure.figsize': (8, 8)}):
        plot_spatial(
        adata=adata_vis,
        # labels to show on a plot
        color=clust_col, labels=clust_labels,
        show_img=True,
        reorder_cmap=(1,4),
        crop_x=(130,1250),
        crop_y=(420,1550),
        # 'fast' (white background) or 'dark_background'
        style='fast',
        # limit color scale at 99.2% quantile of cell abundance
        max_color_quantile=0.992,
        # size of locations (adjust depending on figure size)
        circle_diameter=6,
        colorbar_position='right'
    )    
plt.savefig('Figure3h.pdf')  